## Install Docker

___src: https://docs.docker.com/engine/install/ubuntu/___

1- Uninstall old versions:

The unofficial packages to uninstall are:

docker.io</br>
docker-compose</br>
docker-compose-v2</br>
docker-doc</br>
podman-docker</br>

    for pkg in docker.io docker-doc docker-compose docker-compose-v2 podman-docker containerd runc; do sudo apt-get remove $pkg; done

2- Install from a package:

If you can't use Docker's apt repository to install Docker Engine, you can download the deb file for your release and install it manually. You need to download a new file each time you want to upgrade Docker Engine.

Go to `https://download.docker.com/linux/ubuntu/dists/`.

Select your Ubuntu version in the list.

Go to pool/stable/ and select the applicable architecture (amd64, armhf, arm64, or s390x).

Download the following deb files for the Docker Engine, CLI, containerd, and Docker Compose packages:

    containerd.io_<version>_<arch>.deb
    docker-ce_<version>_<arch>.deb
    docker-ce-cli_<version>_<arch>.deb
    docker-buildx-plugin_<version>_<arch>.deb
    docker-compose-plugin_<version>_<arch>.deb
    
Install the .deb packages. Update the paths in the following example to where you downloaded the Docker packages.

 sudo dpkg -i ./containerd.io_<version>_<arch>.deb \
  ./docker-ce_<version>_<arch>.deb \
  ./docker-ce-cli_<version>_<arch>.deb \
  ./docker-buildx-plugin_<version>_<arch>.deb \
  ./docker-compose-plugin_<version>_<arch>.deb
  
The Docker daemon starts automatically.

Verify that the Docker Engine installation is successful by running the hello-world image.

     sudo service docker start
     sudo docker run hello-world


## Install CUDA

Install CUDA for using GPU

1- sudo apt update</br>
2- sudo apt upgrade</br>
3- sudo apt install ubuntu-drivers-common</br>
4- sudo ubuntu-drivers devices</br>
5- recommends the NVIDIA driver 535</br>

	driver   : nvidia-driver-535 - distro non-free recommended
	
6- sudo apt install nvidia-driver-535</br>
7- Reboot</br>
8- Using NVIDIA icon in top page change to "Switch to: NVIDIA (On-Demand) and then Logout.</br>
9- nvidia-smi</br>

you must see a table. At the top of the table, we will see the driver version and CUDA driver API compatibility:

	NVIDIA-SMI 535.86.05              Driver Version: 535.86.05    CUDA Version: 12.2

10- sudo apt install gcc</br>
11- Install CUDA toolkit Ubuntu</br>

src:https://developer.nvidia.com/cuda-downloads?target_os=Linux&target_arch=x86_64&Distribution=Ubuntu&target_version=22.04&target_type=deb_network

	wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64/cuda-keyring_1.1-1_all.deb
	sudo dpkg -i cuda-keyring_1.1-1_all.deb
	sudo apt-get update  ((for this step you must use VPN such as windscribe:( ))
	sudo apt-get -y install cuda-toolkit-12-3 ((for this step you must use VPN such as windscribe:( ))
	
If you encounter dependency errors during the installation, try running `sudo apt --fix-broken install` to fix them. Apt will suggest running it if needed.

12- Reboot</br>
13- Environment setup</br>

We will now proceed to update the environment variables as recommended by the NVIDIA documentation.
Add the following line to your `.bashrc` file using `nano ~/.bashrc` and paste the following lines at the end of the file.

	export PATH=/usr/local/cuda/bin${PATH:+:${PATH}}
	export LD_LIBRARY_PATH=/usr/local/cuda-12.2/lib64\
                         	 ${LD_LIBRARY_PATH:+:${LD_LIBRARY_PATH}}

Save the file.

14- Reboot
	
15- Test the CUDA toolkit</br>
	
    nvcc -V
	
You must see:

	nvcc: NVIDIA (R) Cuda compiler driver
	Copyright (c) 2005-2023 NVIDIA Corporation
	Built on Fri_Nov__3_17:16:49_PDT_2023
	Cuda compilation tools, release 12.3, V12.3.103
	Build cuda_12.3.r12.3/compiler.33492891_0

src: https://www.cherryservers.com/blog/install-cuda-ubuntu

## Install Ollama

src: https://github.com/jmorganca/ollama/blob/main/docs/linux.md

1- Create a user for Ollama:

    sudo useradd -r -s /bin/false -m -d /usr/share/ollama ollama

2- Create a service file in `/etc/systemd/system/ollama.service`:

    [Unit]
    Description=Ollama Service
    After=network-online.target
    
    [Service]
    ExecStart=/usr/bin/ollama serve
    User=ollama
    Group=ollama
    Restart=always
    RestartSec=3
    
    [Install]
    WantedBy=default.target

3- Then start the service:

    sudo systemctl daemon-reload
    sudo systemctl enable ollama

4- Start Ollama using systemd:

    sudo systemctl start ollama

5- Update ollama by downloading the ollama binary:

    sudo curl -L https://ollama.ai/download/ollama-linux-amd64 -o /usr/bin/ollama
    sudo chmod +x /usr/bin/ollama

6- To view logs of Ollama running as a startup service, run:

    journalctl -u ollama

## Ollama Docker image

src:https://hub.docker.com/r/ollama/ollama

1- docker pull ollama/ollama

2- CPU only:

    docker run -d -v ollama:/root/.ollama -p 11434:11434 --name ollama ollama/ollama

3- Nvidia GPU with Apt

3-1- Configure the repository:

    curl -fsSL https://nvidia.github.io/libnvidia-container/gpgkey \
        | sudo gpg --dearmor -o /usr/share/keyrings/nvidia-container-toolkit-keyring.gpg
    curl -s -L https://nvidia.github.io/libnvidia-container/stable/deb/nvidia-container-toolkit.list \
        | sed 's#deb https://#deb [signed-by=/usr/share/keyrings/nvidia-container-toolkit-keyring.gpg] https://#g' \
        | sudo tee /etc/apt/sources.list.d/nvidia-container-toolkit.list
    sudo apt-get update

3-2- Install the NVIDIA Container Toolkit packages

    sudo apt-get install -y nvidia-container-toolkit

3-3- Configure Docker to use Nvidia driver:

    sudo nvidia-ctk runtime configure --runtime=docker
    sudo systemctl restart docker

3-4- Start the container:

    docker run -d --gpus=all -v ollama:/root/.ollama -p 11434:11434 --name ollama ollama/ollama

3-5- Run model locally

Now you can run a model:

    docker exec -it ollama ollama run llama2 

for the first time this code will download your desired model [Ollama Models](https://ollama.ai/library).

4- If you want run again and again model use below code in terminal and again 2 (for only CPU) and 3-4(GPU):

    sudo docker ps -a
    sudo docker stop ollama
    sudo docker rm ollama

# Latest Method For Ollama Installation with Langchain

[Build your own RAG and run it locally: Langchain + Ollama + Streamlit](https://blog.duy-huynh.com/build-your-own-rag-and-run-them-locally/)


[Install Ollama](src: https://github.com/jmorganca/ollama/blob/main/docs/linux.md#manual-install)

1- Install Ollama in linux:

	sudo curl -L https://ollama.ai/download/ollama-linux-amd64 -o /usr/bin/ollama
	sudo chmod +x /usr/bin/ollama

2- Create a user for Ollama:

	sudo useradd -r -s /bin/false -m -d /usr/share/ollama ollama

3- Create a service file in /etc/systemd/system/ollama.service:

	[Unit]
	Description=Ollama Service
	After=network-online.target

	[Service]
	ExecStart=/usr/bin/ollama serve
	User=ollama
	Group=ollama
	Restart=always
	RestartSec=3

	[Install]
	WantedBy=default.target

4- Then start the service:

	sudo systemctl daemon-reload
	udo systemctl enable ollama
	
5- Start Ollama using systemd:

	sudo systemctl start ollama


============================================

## Above Steps not Worked in Ubuntu

============================================

Using Ollama as Docker (src:https://hub.docker.com/r/ollama/ollama)

1- docker pull ollama/ollama

2- CPU only

	docker run -d -v ollama:/root/.ollama -p 11434:11434 --name ollama ollama/ollama
	
3- Using Ollama with GPU needs at least 4 GPU 4Gb (total 16 Gb), so I can only use CPU.

4- Run model locally

	docker exec -it ollama ollama run llama2

6- Create new env with python < 3.12 

	- micromamba activate base
	- micromamba create -n ollama python=3.11
	- micromamba activate ollama
	
7- Build the RAG pipeline (src:https://blog.duy-huynh.com/build-your-own-rag-and-run-them-locally/)

	- pip install langchain==0.0.343
	- pip install streamlit==1.29.0
	- pip install streamlit-chat==0.1.1
	- pip install pypdf==3.17.1
	- pip install fastembed==0.1.1
	- pip install openai==1.3.6
	- pip install langchainhub==0.1.14
	- pip install chromadb==0.4.18
	- pip install watchdog==3.0.0

8- Some Commands with Ollama

    sudo docker ps -a # List of AvailableModels
    sudo docker stop ollama # Stop the current model
    sudo docker rm ollama # Remove the current model

9- How do I clean the memory cache?

    sync && echo 3 | sudo tee /proc/sys/vm/drop_caches

# Using Streamlit with Ollama and Langchain

Save below code in a file with name myapp.py:

In [ ]:
# Import Required Libraries
import os
import sys
import tempfile
import textwrap
import streamlit as st
from datetime import datetime
from streamlit_chat import message
from langchain.llms import Ollama #Cohere
from langchain.vectorstores import Chroma, FAISS
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA, LLMChain
from langchain.embeddings import HuggingFaceEmbeddings #CohereEmbeddings
from langchain.memory import ConversationBufferMemory
from langchain.document_loaders import PyMuPDFLoader, DirectoryLoader
from langchain.memory.chat_message_histories import StreamlitChatMessageHistory
from langchain.text_splitter import CharacterTextSplitter,RecursiveCharacterTextSplitter

__import__("pysqlite3")
sys.modules["sqlite3"] = sys.modules.pop("pysqlite3")

#--------------------------------------------------------------#
# Setting Up Streamlit Page
st.set_page_config(page_title="Ollama Chatbot", page_icon= "💬")

#--------------------------------------------------------------#
with st.sidebar:
    st.title('💬 OLLAMA Chatbot')
    
    #st.divider()
    # Select the model
    selected_model = st.selectbox('Choose a model', ['Phi-2', 'Llama2', "Orca-mini",
                                                     'Zephyr', 'Code Llama', 'Mistral'],
                                   key='selected_model')

    if selected_model == "Phi-2":
        llm_model = "phi"
        st.caption("""
                   The Phi-2............................
                   """)
    elif selected_model == "Llama2":
        llm_model = "llama2"
        st.caption("""
                   Llama 2 is released by Meta Platforms, Inc.
                   """)
    elif selected_model == "Orca-mini":
        llm_model = "orca-mini"
        st.caption("""
                   Orca-mini..................................
                   """)
    elif selected_model == "Zephyr":
        llm_model = "zephyr"
        st.caption("""
                   Zephyr is a 7 billion parameter model, fine-tuned on Mistral to achieve results similar to 
                   Llama 2 70B Chat in several benchmarks.
                   """)
    elif selected_model == "Code llama":
        llm_model = "codellama"
        st.caption("""
                   Code Llama is a model for generating and discussing code, built on top of Llama 2.
                   """)
    elif selected_model == "Mistral":
        llm_model = "mistral"
        st.caption("""
                   The Mistral 7B model released by Mistral AI.
                   """)
    #st.divider()
    temp_r = st.slider("Temperature", 0.0, 0.9, 0.0, 0.1)
    chunk_size = st.slider("Chunk Size for Splitting Document ", 256, 1024, 400, 10)
    chunk_overlap = st.slider("Chunk Overlap ", 0, 100, 20, 5)
    clear_button = st.button("Clear Conversation", key="clear")

#-----------------------Functions-------------------------------#
# function for loading the embedding model
def load_embedding_model(model_path, normalize_embedding=True):
    return HuggingFaceEmbeddings(
        model_name=model_path,
        model_kwargs={'device': 'cuda'}, #  you can set model_kwargs={'device': 'cuda:0'} for the first GPU, model_kwargs={'device': 'cuda:1'} for the second GPU, and so on.(src:https://github.com/langchain-ai/langchain/issues/10436)
        #model_kwargs={'device':'cpu'}, # here we will run the model with CPU only
        encode_kwargs = {
            'normalize_embeddings': normalize_embedding # keep True to compute cosine similarity
        }
    )

#--------------------------------------------------------------#
# Function for creating embeddings using FAISS
def create_embeddings(chunks, embedding_model, storing_path="vectorstore"):
    # Creating the embeddings using FAISS
    vectorstore = FAISS.from_documents(chunks, embedding_model)
    
    # Saving the model in current directory
    vectorstore.save_local(storing_path)
    
    # returning the vectorstore
    return vectorstore

#--------------------------------------------------------------#
# Creating the chain for Question Answering
def load_qa_chain(retriever, llm, prompt):
    return RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever, # here we are using the vectorstore as a retriever
        chain_type="stuff",
        return_source_documents=True, # including source documents in output
        chain_type_kwargs={'prompt': prompt} # customizing the prompt
    )

#--------------------------------------------------------------#
# tabs
tab1, tab2, tab3, tab4 = st.tabs(["💬 Chatbot", "🖹 ChatPDFs", "📈 ChatPandas", "🌍 ChatMaps"])

#--------------------------------------------------------------#
# Chatbot Tab
# with tab1():

#--------------------------------------------------------------#
# ChatPDFs Tab
with tab2:
	# Upload PDF files
    uploaded_PDF_files = st.file_uploader("Upload multiple files", accept_multiple_files=True, type="pdf")

if uploaded_PDF_files:
    with tempfile.TemporaryDirectory() as tmpdir:
        for uploaded_file in uploaded_PDF_files:
            file_name = uploaded_file.name
            file_content = uploaded_file.read()
            st.write("Filename: ", file_name)

            # Write the content of the PDF files to a temporary directory
            with open(os.path.join(tmpdir, file_name), "wb") as file:
                file.write(file_content)

        # Load the PDF files from the temporary directory
        loader = DirectoryLoader(tmpdir, glob="**/*.pdf", loader_cls=PyMuPDFLoader)
        documents = loader.load()

        # Split the PDF files into smaller chunks of text
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
        documents = text_splitter.split_documents(documents)
        embeddings = load_embedding_model(model_path="all-MiniLM-L6-v2")
        vectorstore = Chroma.from_documents(documents, embeddings)
        #vectorstore.persist()
        retriever = vectorstore.as_retriever()

        prompt_template = """ 
        System Prompt:
        Your are an AI chatbot that helps users chat with PDF documents. How may I help you today?

        {context}

        {question}
        """
        PROMPT = PromptTemplate(
        template=prompt_template, input_variables=["context", "question"]
        )
        chain_type_kwargs = {"prompt": PROMPT}

        chain = RetrievalQA.from_chain_type(
        llm=Ollama(model=llm_model, temperature=temp_r),
        chain_type="stuff",
        retriever=retriever,
        chain_type_kwargs=chain_type_kwargs,
        )
        # Question-Answer
        # Get the user question
        query = st.text_input("Ask a question:")

        if query:
                response = chain({'query': query})
                # Wrapping the text for better output in Jupyter Notebook
                wrapped_text = textwrap.fill(response['result'], width=100)
                # Display the answer
                st.markdown(f"**Q:** {query}")
                st.markdown(f"**A:** {wrapped_text}")

#--------------------------------------------------------------#
# ChatPandas Tab
# with tab3():

#--------------------------------------------------------------#
# ChatMaps Tab
# with tab4():


After Run Ollama docker run below code:

    streamlit run myapp.py

============================================

## Important Note:

In 2023-Dec-20, Ollama pulled `Phi-2` model. This `Phi-2 Model` is a Small Language Model (SLM) type and released by `Microsoft` In 2023-Dec-15. This small model is very incredible model that with 1.6GB and 2.7B parameter is very `????????????????`  .

For use this model. I first update `Ollama Docker` using `????????????????`.

After that:

    sudo docker stop ollama
    sudo docker rm ollama

Because `Phi-2 model` is small, so I can use `CUDA` with this model. So I can active `GPU` using:

    docker run -d --gpus=all -v ollama:/root/.ollama -p 11434:11434 --name ollama ollama/ollama

** I remeber that for other models such as `llama2, Orca-mini, Zephyr, Code Llama, Mistral and other 7B models` I only use `CPU` using:

    docker run -d -v ollama:/root/.ollama -p 11434:11434 --name ollama ollama/ollama

After activing the `GPU` or `CPU` I can run `Streamlit` for using LLMs.

============================================

## Building Own LLM model using GGUF from Huggingface

Refer Ollama Github

huggingface-cli download TheBloke/MistralLite-7B-GGUF mistrallite.Q4_K_M.gguf --local-dir downloads --local-dir-use-symlinks False


huggingface-cli download kroonen/phi-2-GGUF phi-2_Q4_K_M.gguf --local-dir downloads --local-dir-use-symlinks False


huggingface-cli download TheBloke/Mistral-7B-OpenOrca-GGUF mistral-7b-openorca.Q4_K_M.gguf --local-dir . --local-dir-use-symlinks False

# New ChatPDF with Streamlit-Langchain

In [ ]:
import os
import tempfile
import streamlit as st
#from langchain.chat_models import ChatOpenAI
from langchain.llms import Ollama
from langchain.document_loaders import PyPDFLoader
from langchain.memory import ConversationBufferMemory
from langchain.memory.chat_message_histories import StreamlitChatMessageHistory
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.callbacks.base import BaseCallbackHandler
from langchain.chains import ConversationalRetrievalChain
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.text_splitter import RecursiveCharacterTextSplitter

#--------------------------------------------------------------#
st.set_page_config(page_title="Chat with Documents", page_icon="📖")
st.title("📚: Chat with Documents")

#--------------------------------------------------------------#
MODES = ("CPU", "GPU")

#--------------------------------------------------------------#
with st.sidebar:
    st.title('💬 OLLAMA Chatbot')

    # Select Processing Mode
    mode = st.radio("Choose a mode", MODES)
    if mode == "CPU":
        device = "cpu"
    elif mode == "GPU":
        device = "cuda"
    else:
        device = "cpu"

    # Select the model
    model_name = st.selectbox('Choose a model: ', ['Phi-2', 'Llama2', "Orca-mini",
                                                 'Zephyr', 'Code Llama', 'Mistral'],
                              key='model_name')

    if model_name == "Phi-2":
        llm_model = "phi"
        st.caption("""
                   Phi-2: a 2.7B language model by Microsoft Research
				   that demonstrates outstanding reasoning and language understanding capabilities.
                   """)
    elif model_name == "Llama2":
        llm_model = "llama2"
        st.caption("""
                   Llama 2 is released by Meta Platforms, Inc.
                   """)
    elif model_name == "Orca-mini":
        llm_model = "orca-mini"
        st.caption("""
                   A general-purpose model ranging from 3 billion parameters to 70 billion, suitable for entry-level hardware.
                   """)
    elif model_name == "Zephyr":
        llm_model = "zephyr"
        st.caption("""
                   Zephyr is a 7 billion parameter model, fine-tuned on Mistral to achieve results similar to 
                   Llama 2 70B Chat in several benchmarks.
                   """)
    elif model_name == "Code llama":
        llm_model = "codellama"
        st.caption("""
                   Code Llama is a model for generating and discussing code, built on top of Llama 2.
                   """)
    elif model_name == "Mistral":
        llm_model = "mistral"
        st.caption("""
                   The Mistral 7B model released by Mistral AI.
                   """)
    #st.divider()
    temp_r = st.slider("Temperature", 0.0, 0.9, 0.0, 0.1)
    chunk_size = st.slider("Chunk Size for Splitting Document ", 200, 3000, 1500, 20)
    chunk_overlap = st.slider("Chunk Overlap ", 0, 500, 200, 10)

#--------------------------------------------------------------#
@st.cache_resource(ttl="1h")
def configure_retriever(uploaded_files):
    # Read documents
    docs = []
    temp_dir = tempfile.TemporaryDirectory()
    for file in uploaded_files:
        temp_filepath = os.path.join(temp_dir.name, file.name)
        with open(temp_filepath, "wb") as f:
            f.write(file.getvalue())
        loader = PyPDFLoader(temp_filepath)
        docs.extend(loader.load())

    # Split documents
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    splits = text_splitter.split_documents(docs)

    # Create embeddings and store in vectordb
    embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2",
                                       model_kwargs={'device': device},
                                       encode_kwargs = {'normalize_embeddings': True # keep True to compute cosine similarity
                                                        })
    vectordb = DocArrayInMemorySearch.from_documents(splits, embeddings)

    # Define retriever
    retriever = vectordb.as_retriever(search_type="mmr", search_kwargs={"k": 2, "fetch_k": 4})

    return retriever

#--------------------------------------------------------------#
class StreamHandler(BaseCallbackHandler):
    def __init__(self, container: st.delta_generator.DeltaGenerator, initial_text: str = ""):
        self.container = container
        self.text = initial_text
        self.run_id_ignore_token = None

    def on_llm_start(self, serialized: dict, prompts: list, **kwargs):
        # Workaround to prevent showing the rephrased question as output
        if prompts[0].startswith("Human"):
            self.run_id_ignore_token = kwargs.get("run_id")

    def on_llm_new_token(self, token: str, **kwargs) -> None:
        if self.run_id_ignore_token == kwargs.get("run_id", False):
            return
        self.text += token
        self.container.markdown(self.text)

#--------------------------------------------------------------#
class PrintRetrievalHandler(BaseCallbackHandler):
    def __init__(self, container):
        self.status = container.status("**Context Retrieval**")

    def on_retriever_start(self, serialized: dict, query: str, **kwargs):
        self.status.write(f"**Question:** {query}")
        self.status.update(label=f"**Context Retrieval:** {query}")

    def on_retriever_end(self, documents, **kwargs):
        for idx, doc in enumerate(documents):
            source = os.path.basename(doc.metadata["source"])
            self.status.write(f"**Document {idx} from {source}**")
            self.status.markdown(doc.page_content)
        self.status.update(state="complete")
#--------------------------------------------------------------#
# Upload PDF Files
uploaded_files = st.file_uploader(
    label="Upload PDF files", type=["pdf"], accept_multiple_files=True
)
if not uploaded_files:
    st.info("Please upload PDF documents to continue.")
    st.stop()

retriever = configure_retriever(uploaded_files)

#--------------------------------------------------------------#
# Setup memory for contextual conversation
msgs = StreamlitChatMessageHistory()
memory = ConversationBufferMemory(memory_key="chat_history", chat_memory=msgs, return_messages=True)

# Setup LLM and QA chain
llm = Ollama(model=llm_model, temperature=temp_r)#, streaming=True)
qa_chain = ConversationalRetrievalChain.from_llm(
    llm, retriever=retriever, memory=memory, verbose=True
)

#--------------------------------------------------------------#
if len(msgs.messages) == 0 or st.sidebar.button("Clear message history"):
    msgs.clear()
    msgs.add_ai_message("How can I help you?")

avatars = {"human": "user", "ai": "assistant"}

#--------------------------------------------------------------#
for msg in msgs.messages:
    st.chat_message(avatars[msg.type]).write(msg.content)

#--------------------------------------------------------------#
if user_query := st.chat_input(placeholder="Ask me anything!"):
    st.chat_message("user").write(user_query)

    with st.chat_message("assistant"):
        retrieval_handler = PrintRetrievalHandler(st.container())
        stream_handler = StreamHandler(st.empty())
        response = qa_chain.run(user_query, callbacks=[retrieval_handler, stream_handler])

# Free Up GPU

    nvidia-smi
    
    sudo fuser -v /dev/nvidia*

    sudo kill -9 PID

    nvidia-smi

# Chat Maps(NetCDF)

In [ ]:
import os
import json
#import yaml
import folium
import requests
import numpy as np
import pandas as pd
import xarray as xr
import streamlit as st
import geopandas as gpd
from pyproj import Geod
from langchain.llms import Ollama
from langchain.chains import LLMChain
from geopy.geocoders import Nominatim
from streamlit_folium import st_folium
from requests.exceptions import Timeout
from streamlit_folium import folium_static ################
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

from langchain.callbacks.base import BaseCallbackHandler

st.markdown(
            f'''
            <style>
                .reportview-container .sidebar-content {{
                    padding-top: {0.1}rem;
                }}
                .reportview-container .main .block-container {{
                    padding-top: {1}rem;
                }}
            </style>
            ''',unsafe_allow_html=True)
#-----------------------------------------------------------------#
data_path = "./data/"
coastline_shapefile = "./data/natural_earth/coastlines/ne_50m_coastline.shp"
clicked_coords = None

#-----------------------------------------------------------------#
system_role = """
You are the system that should help people to evaluate the impact of climate change
on decisions they are taking today (e.g. install wind turbines, solar panels, build a building,
parking lot, open a shop, buy crop land). You are working with data on a local level,
and decisions also should be given for particular locations. You will be given information 
about changes in environmental variables for particular location, and how they will 
change in a changing climate. Your task is to provide assessment of potential risks 
and/or benefits for the planned activity related to change in climate. Use information 
about the country to retrieve information about policies and regulations in the 
area related to climate change, environmental use and activity requested by the user.
You don't have to use all variables provided to you, if the effect is insignificant,
don't use variable in analysis. DON'T just list information about variables, don't 
just repeat what is given to you as input. I don't want to get the code, 
I want to receive a narrative, with your assessments and advice. Format 
your response as MARKDOWN, don't use Heading levels 1 and 2.
"""

content_message = "{user_message} \n \
      Location: latitude = {lat}, longitude = {lon} \
      Adress: {location_str} \
      Policy: {policy} \
      Distance to the closest coastline: {distance_to_coastline} \
      Elevation above sea level: {elevation} \
      Current landuse: {current_land_use} \
      Current soil type: {soil} \
      Current mean monthly temperature for each month: {hist_temp_str} \
      Future monthly temperatures for each month at the location: {future_temp_str}\
      Curent precipitation flux (mm/month): {hist_pr_str} \
      Future precipitation flux (mm/month): {future_pr_str} \
      Curent u wind component (in m/s): {hist_uas_str} \
      Future u wind component (in m/s): {future_uas_str} \
      Curent v wind component (in m/s): {hist_vas_str} \
      Future v wind component (in m/s): {future_vas_str} \
      "

#-----------------------------------------------------------------#
class StreamHandler(BaseCallbackHandler):
    """
    Taken from here: https://discuss.streamlit.io/t/langchain-stream/43782
    """

    def __init__(self, container, initial_text="", display_method="markdown"):
        self.container = container
        self.text = initial_text
        self.display_method = display_method

    def on_llm_new_token(self, token: str, **kwargs) -> None:
        self.text += token
        display_function = getattr(self.container, self.display_method, None)
        if display_function is not None:
            display_function(self.text)
        else:
            raise ValueError(f"Invalid display_method: {self.display_method}")

#-----------------------------------------------------------------#
@st.cache_data
def get_location(lat, lon):
    """
    Returns the address of a given latitude and longitude using the Nominatim geocoding service.

    Parameters:
    lat (float): The latitude of the location.
    lon (float): The longitude of the location.

    Returns:
    dict: A dictionary containing the address information of the location.
    """
    geolocator = Nominatim(user_agent="climsight")
    location = geolocator.reverse((lat, lon), language="en")
    return location.raw["address"]

#-----------------------------------------------------------------#
@st.cache_data
def get_adress_string(location):
    """
    Returns a tuple containing two strings:
    1. A string representation of the location address with all the key-value pairs in the location dictionary.
    2. A string representation of the location address with only the country, state, city and road keys in the location dictionary.

    Parameters:
    location (dict): A dictionary containing the location address information.

    Returns:
    tuple: A tuple containing two strings.
    """
    location_str = "Adress: "
    for key in location:
        location_str = location_str + f"{key}:{location[key]}, "
    location_str_for_print = "**Address:** "
    if "country" in location:
        location_str_for_print += f"{location['country']}, "
    if "state" in location:
        location_str_for_print += f"{location['state']}, "
    if "city" in location:
        location_str_for_print += f"{location['city']}, "
    if "road" in location:
        location_str_for_print += f"{location['road']}"
    return location_str, location_str_for_print

#-----------------------------------------------------------------#
@st.cache_data
def closest_shore_distance(lat: float, lon: float, coastline_shapefile: str) -> float:
    """
    Calculates the closest distance between a given point (lat, lon) and the nearest point on the coastline.

    Args:
        lat (float): Latitude of the point
        lon (float): Longitude of the point
        coastline_shapefile (str): Path to the shapefile containing the coastline data

    Returns:
        float: The closest distance between the point and the coastline, in meters.
    """
    geod = Geod(ellps="WGS84")
    min_distance = float("inf")

    coastlines = gpd.read_file(coastline_shapefile)

    for _, row in coastlines.iterrows():
        geom = row["geometry"]
        if geom.geom_type == "MultiLineString":
            for line in geom.geoms:
                for coastal_point in line.coords:
                    _, _, distance = geod.inv(
                        lon, lat, coastal_point[0], coastal_point[1]
                    )
                    min_distance = min(min_distance, distance)
        else:  # Assuming LineString
            for coastal_point in geom.coords:
                _, _, distance = geod.inv(lon, lat, coastal_point[0], coastal_point[1])
                min_distance = min(min_distance, distance)

    return min_distance

#-----------------------------------------------------------------#
@st.cache_data
def get_elevation_from_api(lat, lon):
    """
    Get the elevation of a location using the Open Topo Data API.

    Parameters:
    lat (float): The latitude of the location.
    lon (float): The longitude of the location.

    Returns:
    float: The elevation of the location in meters.
    """
    url = f"https://api.opentopodata.org/v1/etopo1?locations={lat},{lon}"
    response = requests.get(url)
    data = response.json()
    return data["results"][0]["elevation"]

#-----------------------------------------------------------------#
@st.cache_data
def fetch_land_use(lon, lat):
    """
    Fetches land use data for a given longitude and latitude using the Overpass API.

    Args:
    - lon (float): The longitude of the location to fetch land use data for.
    - lat (float): The latitude of the location to fetch land use data for.

    Returns:
    - data (dict): A dictionary containing the land use data for the specified location.
    """
    overpass_url = "http://overpass-api.de/api/interpreter"
    overpass_query = f"""
    [out:json];
    is_in({lat},{lon})->.a;
    area.a["landuse"];
    out tags;
    """
    response = requests.get(overpass_url, params={"data": overpass_query})
    data = response.json()
    return data

#-----------------------------------------------------------------#
@st.cache_data
def get_soil_from_api(lat, lon):
    """
    Retrieves the soil type at a given latitude and longitude using the ISRIC SoilGrids API.

    Parameters:
    lat (float): The latitude of the location.
    lon (float): The longitude of the location.

    Returns:
    str: The name of the World Reference Base (WRB) soil class at the given location.
    """
    try:
        url = f"https://rest.isric.org/soilgrids/v2.0/classification/query?lon={lon}&lat={lat}&number_classes=5"
        response = requests.get(url, timeout=2)  # Set timeout to 2 seconds
        data = response.json()
        return data["wrb_class_name"]
    except Timeout:
        return "not found"

#-----------------------------------------------------------------#
@st.cache_data
def load_data():
    hist = xr.open_mfdataset(f"{data_path}/AWI_CM_mm_historical*.nc", compat="override")
    future = xr.open_mfdataset(f"{data_path}/AWI_CM_mm_ssp585*.nc", compat="override")
    return hist, future

#-----------------------------------------------------------------#
def convert_to_mm_per_month(monthly_precip_kg_m2_s1):
    days_in_months = np.array([31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31])
    return monthly_precip_kg_m2_s1 * 60 * 60 * 24 * days_in_months

#-----------------------------------------------------------------#
@st.cache_data
def extract_climate_data(lat, lon, _hist, _future):
    """
    Extracts climate data for a given latitude and longitude from historical and future datasets.

    Args:
    - lat (float): Latitude of the location to extract data for.
    - lon (float): Longitude of the location to extract data for.
    - _hist (xarray.Dataset): Historical climate dataset.
    - _future (xarray.Dataset): Future climate dataset.

    Returns:
    - df (pandas.DataFrame): DataFrame containing present day and future temperature, precipitation, and wind speed data for each month of the year.
    - data_dict (dict): Dictionary containing string representations of the extracted climate data.
    """
    hist_temp = hist.sel(lat=lat, lon=lon, method="nearest")["tas"].values - 273.15
    hist_temp_str = np.array2string(hist_temp.ravel(), precision=3, max_line_width=100)[
        1:-1
    ]

    hist_pr = hist.sel(lat=lat, lon=lon, method="nearest")["pr"].values
    hist_pr = convert_to_mm_per_month(hist_pr)

    hist_pr_str = np.array2string(hist_pr.ravel(), precision=3, max_line_width=100)[
        1:-1
    ]

    hist_uas = hist.sel(lat=lat, lon=lon, method="nearest")["uas"].values
    hist_uas_str = np.array2string(hist_uas.ravel(), precision=3, max_line_width=100)[
        1:-1
    ]

    hist_vas = hist.sel(lat=lat, lon=lon, method="nearest")["vas"].values
    hist_vas_str = np.array2string(hist_vas.ravel(), precision=3, max_line_width=100)[
        1:-1
    ]

    future_temp = future.sel(lat=lat, lon=lon, method="nearest")["tas"].values - 273.15
    future_temp_str = np.array2string(
        future_temp.ravel(), precision=3, max_line_width=100
    )[1:-1]

    future_pr = future.sel(lat=lat, lon=lon, method="nearest")["pr"].values
    future_pr = convert_to_mm_per_month(future_pr)
    future_pr_str = np.array2string(future_pr.ravel(), precision=3, max_line_width=100)[
        1:-1
    ]

    future_uas = future.sel(lat=lat, lon=lon, method="nearest")["uas"].values
    future_uas_str = np.array2string(
        future_uas.ravel(), precision=3, max_line_width=100
    )[1:-1]

    future_vas = future.sel(lat=lat, lon=lon, method="nearest")["vas"].values
    future_vas_str = np.array2string(
        future_vas.ravel(), precision=3, max_line_width=100
    )[1:-1]
    df = pd.DataFrame(
        {
            "Present day Temperature": hist_temp[0, 0, :],
            "Future Temeprature": future_temp[0, 0, :],
            "Present day Precipitation": hist_pr[0, 0, :],
            "Future Precipitation": future_pr[0, 0, :],
            "Present day Wind speed": np.hypot(hist_uas[0, 0, :], hist_vas[0, 0, :]),
            "Future Wind speed": np.hypot(future_uas[0, 0, :], future_vas[0, 0, :]),
            "Month": range(1, 13),
        }
    )
    data_dict = {
        "hist_temp": hist_temp_str,
        "hist_pr": hist_pr_str,
        "hist_uas": hist_uas_str,
        "hist_vas": hist_vas_str,
        "future_temp": future_temp_str,
        "future_pr": future_pr_str,
        "future_uas": future_uas_str,
        "future_vas": future_vas_str,
    }
    return df, data_dict

#-----------------------------------------------------------------#
hist, future = load_data()

#-----------------------------------------------------------------#
st.title(
    " :deciduous_tree:  Environmental Assessment"
) 
# Emoji list: https://www.fileformat.info/info/emoji/list.htm
#" :cyclone: :ocean: :globe_with_meridians:  Climate Foresight"
# :umbrella_with_rain_drops: :earth_africa:  :tornado:
user_message = st.text_input(
    "Describe activity you would like to evaluate for this location:"
)

#-----------------------------------------------------------------#
col1, col2 = st.columns(2)
lat_default = 33.2583
lon_default = 51.3081

m = folium.Map(location=[lat_default, lon_default], zoom_start=9)
#--------------------------------------------------------------#
MODES = ("CPU", "GPU")

#-----------------------------------------------------------------#
with st.sidebar:
    st.info(
    "Click on your desired location")
    
    map_data = st_folium(m, width = 300, height=300)

    if map_data:
        clicked_coords = map_data["last_clicked"]
        if clicked_coords:
            lat_default = clicked_coords["lat"]
            lon_default = clicked_coords["lng"]

    lat = col1.number_input("Latitude", value=lat_default, format="%.4f")
    lon = col2.number_input("Longitude", value=lon_default, format="%.4f")

    # Select Processing Mode
    mode = st.radio("Choose a mode", MODES)
    if mode == "CPU":
        device = "cpu"
    elif mode == "GPU":
        device = "cuda"
    else:
        device = "cpu"

    # Select the model
    model_name = st.selectbox('Choose a model: ', ['Phi-2', 'Llama2', "Orca-mini",
                                                 'Zephyr', 'Code Llama', 'Mistral'],
                              key='model_name')

    if model_name == "Phi-2":
        llm_model = "phi"
        st.caption("""
                   Phi-2: a 2.7B language model by Microsoft Research
				   that demonstrates outstanding reasoning and language understanding capabilities.
                   """)
    elif model_name == "Llama2":
        llm_model = "llama2"
        st.caption("""
                   Llama 2 is released by Meta Platforms, Inc.
                   """)
    elif model_name == "Orca-mini":
        llm_model = "orca-mini"
        st.caption("""
                   A general-purpose model ranging from 3 billion parameters to 70 billion, suitable for entry-level hardware.
                   """)
    elif model_name == "Zephyr":
        llm_model = "zephyr"
        st.caption("""
                   Zephyr is a 7 billion parameter model, fine-tuned on Mistral to achieve results similar to 
                   Llama 2 70B Chat in several benchmarks.
                   """)
    elif model_name == "Code llama":
        llm_model = "codellama"
        st.caption("""
                   Code Llama is a model for generating and discussing code, built on top of Llama 2.
                   """)
    elif model_name == "Mistral":
        llm_model = "mistral"
        st.caption("""
                   The Mistral 7B model released by Mistral AI.
                   """)
    folium.Marker(location=[lat, lon]).add_to(m)

#-----------------------------------------------------------------#
if st.button("Generate") and user_message:
    with st.spinner("Getting info on a point..."):
        location = get_location(lat, lon)
        location_str, location_str_for_print = get_adress_string(location)
        st.markdown(f"**Coordinates:** {round(lat, 4)}, {round(lon, 4)}")
        st.markdown(location_str_for_print)
        elevation = get_elevation_from_api(lat, lon)
        st.markdown(f"**Elevation:** {elevation} m")

        land_use_data = fetch_land_use(lon, lat)
        try:
            current_land_use = land_use_data["elements"][0]["tags"]["landuse"]
        except:
            current_land_use = "Not known"
        st.markdown(f"**Current land use:** {current_land_use}")

        soil = get_soil_from_api(lat, lon)
        st.markdown(f"**Soil type:** {soil}")

        distance_to_coastline = closest_shore_distance(lat, lon, coastline_shapefile)
        st.markdown(f"**Distance to the shore:** {round(distance_to_coastline, 2)} m")

        # create pandas dataframe
        df, data_dict = extract_climate_data(lat, lon, hist, future)
        # Plot the chart
        st.text(
            "Near surface temperature [souce: AWI-CM-1-1-MR, historical and SSP5-8.5]",
        )
        st.line_chart(
            df,
            x="Month",
            y=["Present day Temperature", "Future Temeprature"],
            color=["#d62728", "#2ca02c"],
        )
        st.text(
            "Precipitation [souce: AWI-CM-1-1-MR, historical and SSP5-8.5]",
        )
        st.line_chart(
            df,
            x="Month",
            y=["Present day Precipitation", "Future Precipitation"],
            color=["#d62728", "#2ca02c"],
        )
        st.text(
            "Wind speed [souce: AWI-CM-1-1-MR, historical and SSP5-8.5]",
        )
        st.line_chart(
            df,
            x="Month",
            y=["Present day Wind speed", "Future Wind speed"],
            color=["#d62728", "#2ca02c"],
        )
    policy = ""
    with st.spinner("Generating..."):
        chat_box = st.empty()
        stream_handler = StreamHandler(chat_box, display_method="write")
        llm = Ollama(model=llm_model, temperature=0, callbacks=[stream_handler],)

        system_message_prompt = SystemMessagePromptTemplate.from_template(system_role)
        human_message_prompt = HumanMessagePromptTemplate.from_template(content_message)
        chat_prompt = ChatPromptTemplate.from_messages(
            [system_message_prompt, human_message_prompt]
        )
        chain = LLMChain(
            llm=llm,
            prompt=chat_prompt,
            output_key="review",
            verbose=True,
        )

        output = chain.run(
            user_message=user_message,
            lat=str(lat),
            lon=str(lon),
            location_str=location_str,
            policy=policy,
            distance_to_coastline=str(distance_to_coastline),
            elevation=str(elevation),
            current_land_use=current_land_use,
            soil=soil,
            hist_temp_str=data_dict["hist_temp"],
            future_temp_str=data_dict["future_temp"],
            hist_pr_str=data_dict["hist_pr"],
            future_pr_str=data_dict["future_pr"],
            hist_uas_str=data_dict["hist_uas"],
            future_uas_str=data_dict["future_uas"],
            hist_vas_str=data_dict["hist_vas"],
            future_vas_str=data_dict["future_vas"],
            verbose=True,
        )


Src: https://github.com/koldunovn/climsight/tree/main